# Select a Kernel

Set the kernel to 'Python 3.10 - SDK v2' and then select the 'Authenticate' button (if shown). 

Update the values for: 

 - SUBSCRIPTION
 - RESOURCE_GROUP
 - WS_NAME

The values can be obtained using the dropdown menu. 

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import os

os.environ["APPLICATIONINSIGHTS_STATSBEAT_DISABLED_ALL"] = "true"

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION = "<SUBSCRIPTION_ID>"
RESOURCE_GROUP = "<RESOURCE_GROUP>"
WS_NAME = "<AML_WORKSPACE_NAME>"
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

## Verifiy that a connection can be made

In [ ]:
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location, ":", ws.resource_group)